#### threading

---

#### Event

In [ ]:
from threading import Event, Thread
import time


def countdown(number: int, event: Event) -> None:
    print(f"Countdown from {number} started")
    for index in range(number):
        print(f"Countdown: {number - index}")
        time.sleep(1)
    print(f"Countdown from {number} finished")
    event.set()


event = Event()

thread = Thread(target=countdown, args=(5, event))
thread.start()

event.wait()
print("Countdown completed, main thread continues")

event.clear()

Countdown from 5 started
Countdown: 5
Countdown: 4
Countdown: 3
Countdown: 2
Countdown: 1
Countdown from 5 finished
Countdown completed, main thread continues


#### Condition

In [9]:
from threading import Condition, Thread
import time


def produce(condition: Condition, items: list) -> None:
    for item in range(5):
        with condition:  # ✅ 每次生产时短暂持有锁
            items.append(item)
            print(f"Produced {item}")
            condition.notify()
        time.sleep(2)


def consume(condition: Condition, items: list) -> None:
    while True:  # ✅ 持续消费
        with condition:
            while not items:
                print("等待生产...")
                condition.wait()
            item = items.pop(0)
            print(f"Consumed {item}")


items = []
condition = Condition()

producer_thread = Thread(target=produce, args=(condition, items))
consumer_thread = Thread(target=consume, args=(condition, items))

producer_thread.start()
consumer_thread.start()

producer_thread.join()

Produced 0
Consumed 0
等待生产...
Produced 1
Consumed 1
等待生产...
Produced 2
Consumed 2
等待生产...
Produced 3
Consumed 3
等待生产...
Produced 4
Consumed 4
等待生产...


#### Lock

In [ ]:
from threading import Lock, Thread
import time


def process(source: any, lock: Lock) -> None:
    with lock:
        print(f"Processing {source}")
        time.sleep(5)
        print(f"Finished processing {source}")


def get(source: any, lock: Lock) -> None:
    while True:
        if lock.locked():
            print(f"Wait {source}")
            time.sleep(1)
        else:
            print(f"Get {source}")
            break


lock = Lock()
source = "Data Source"
process_thread = Thread(target=process, args=(source, lock))
get_thread = Thread(target=get, args=(source, lock))
process_thread.start()
get_thread.start()
process_thread.join()
get_thread.join()

Processing Data Source
Wait Data Source
Wait Data Source
Wait Data Source
Wait Data Source
Wait Data Source
Finished processing Data Source
Get Data Source


#### RLock

In [ ]:
from threading import RLock, Thread
import time


def process_data(source: str, lock: RLock) -> None:
    with lock:  # 第一次获取锁
        print(f"🔒 [Process] 开始处理 {source}")
        time.sleep(1)

        # 嵌套调用需要同一把锁的函数
        save_result(source, lock)  # ⭐ 这里需要再次获取锁
        print(f"✅ [Process] 完成处理 {source}")


def save_result(source: str, lock: RLock) -> None:
    with lock:  # 第二次获取锁（RLock 允许同一线程重复获取）
        print(f"💾 [Save] 保存 {source} 的结果")
        time.sleep(0.5)


# 使用 RLock 代替 Lock
lock = RLock()
source = "关键数据"

# 启动线程
t = Thread(target=process_data, args=(source, lock))
t.start()
t.join()

🔒 [Process] 开始处理 关键数据
💾 [Save] 保存 关键数据 的结果
✅ [Process] 完成处理 关键数据


#### Lock/RLock

|特性|`Lock` (普通锁)|`RLock` (可重入锁)|
|---|---|---|
|**重入性**|❌ 同一线程无法重复获取锁|✅ 同一线程可多次获取锁|
|**锁的持有者**|不记录持有线程|记录持有线程和递归深度|
|**释放机制**|必须由获取锁的线程释放|必须由获取锁的线程释放，且释放次数需匹配|
|**性能**|更高（简单实现）|略低（需要维护递归计数）|
|**适用场景**|简单的互斥操作|嵌套/递归调用的代码|

#### 安全使用锁

- 所有线程获取锁的顺序要一致
- 使用超时机制 `acquire(timeout=3)`

#### 同时按照顺序获取多个锁的简洁写法

```python
with lock_a, lock_b:
    # 等同于 with lock_a: with lock_b:
    ...
```

#### Barrier

In [6]:
import threading
import time
import random


# 定义阶段数量（每个阶段一个 Barrier）
phase1_barrier = threading.Barrier(
    parties=3, action=lambda: print("\n== 所有线程完成数据加载，开始清洗 ==\n")
)
phase2_barrier = threading.Barrier(
    parties=3, action=lambda: print("\n== 所有线程完成数据清洗，开始分析 ==\n")
)


def data_processing(thread_id):
    # 第一阶段：数据加载
    load_time = random.uniform(2, 4)
    time.sleep(load_time)
    print(f"线程 {thread_id} 数据加载完成（耗时 {load_time:.1f}s）")

    phase1_barrier.wait()  # 等待所有线程完成加载

    # 第二阶段：数据清洗
    clean_time = random.uniform(2, 4)
    time.sleep(clean_time)
    print(f"线程 {thread_id} 数据清洗完成（耗时 {clean_time:.1f}s）")

    phase2_barrier.wait()  # 等待所有线程完成清洗

    # 第三阶段：数据分析
    analysis_time = random.uniform(2, 4)
    time.sleep(analysis_time)
    print(f"线程 {thread_id} 数据分析完成（耗时 {analysis_time:.1f}s）")


# 创建 3 个线程
threads = []
for i in range(1, 4):
    t = threading.Thread(target=data_processing, args=(i,))
    threads.append(t)
    t.start()

# 等待所有线程完成
for t in threads:
    t.join()

print("所有处理阶段完成！")

线程 2 数据加载完成（耗时 2.5s）
线程 1 数据加载完成（耗时 2.6s）
线程 3 数据加载完成（耗时 2.9s）

== 所有线程完成数据加载，开始清洗 ==

线程 3 数据清洗完成（耗时 2.3s）
线程 1 数据清洗完成（耗时 3.0s）
线程 2 数据清洗完成（耗时 3.5s）

== 所有线程完成数据清洗，开始分析 ==

线程 3 数据分析完成（耗时 2.3s）
线程 2 数据分析完成（耗时 3.6s）
线程 1 数据分析完成（耗时 3.7s）
所有处理阶段完成！


#### Semaphore

In [17]:
import threading
import time
import random

# 初始化一个信号量，允许最多 3 个线程同时访问
semaphore = threading.Semaphore(3)


def access_resource(thread_id):
    with semaphore:  # 请求信号量，如果信号量计数大于 0，线程才能继续
        print(f"⌛️ Thread-{thread_id} has acquired the resource.")
        time.sleep(random.uniform(2, 4))  # 模拟线程使用资源的时间
        print(f"✅ Thread-{thread_id} has released the resource.")


# 创建并启动 5 个线程
threads = []
for i in range(5):
    t = threading.Thread(target=access_resource, args=(i,))
    t.start()
    threads.append(t)

# 等待所有线程完成
for t in threads:
    t.join()

print("All threads have finished.")

⌛️ Thread-0 has acquired the resource.⌛️ Thread-1 has acquired the resource.
⌛️ Thread-2 has acquired the resource.

✅ Thread-0 has released the resource.
⌛️ Thread-3 has acquired the resource.
✅ Thread-2 has released the resource.
⌛️ Thread-4 has acquired the resource.
✅ Thread-1 has released the resource.
✅ Thread-3 has released the resource.
✅ Thread-4 has released the resource.
All threads have finished.


#### BoundedSemaphore

In [ ]:
import threading


bounded_semaphore = threading.BoundedSemaphore(3)

try:
    bounded_semaphore.release()
except ValueError as e:
    print(f"Semaphore released failed, {e}")

try:
    bounded_semaphore.acquire(timeout=0.1)
    bounded_semaphore.release()
    print("Semaphore released successfully")
except ValueError as e:
    print(e)

Semaphore released failed, Semaphore released too many times
Semaphore released successfully


#### Semaphore/BoundedSeamphore

|特性|`Semaphore`|`BoundedSemaphore`|
|---|---|---|
|**最大信号量数**|可以随意增加释放次数，不会抛出异常|不允许释放次数超过初始化的信号量数|
|**`release()` 的限制**|没有限制，可以超过初始计数|`release()` 次数要与 `acquire()` 一致|
|**适用场景**|适合对释放次数没有严格要求的情况|适合需要严格控制释放次数的场景，如防止程序中释放信号量过多导致的问题|

#### Timer

In [1]:
import time
import threading


def do_something(name: any, queue: list):
    queue.append(name)


pool = list()
timer = threading.Timer(2, do_something, args=("Alice", pool))
timer.start()

while not pool:
    print("Waiting for Alice...")
    time.sleep(1)
else:
    print(pool)

Waiting for Alice...
Waiting for Alice...
['Alice']


#### communication tools

---

#### Queue

- `queue.Queue` 普通的 FIFO 队列
- `queue.LifoQueue` LIFO 队列（后进先出）
- `queue.PriorityQueue` 带优先级的队列

In [16]:
import threading
import queue
import time


def produce(production: queue.LifoQueue):
    for goods in range(5):
        time.sleep(1)
        print(f"生产者生产了数据: {goods}")
        production.put(goods)


def consume(production: queue.Queue):
    while True:
        item = production.get()
        if item is None:
            break
        print(f"消费者消费了数据: {item}")
        production.task_done()


production = queue.LifoQueue()
producer_thread = threading.Thread(target=produce, args=(production,))
producer_thread.start()
consumer_thread = threading.Thread(target=consume, args=(production,))
consumer_thread.start()
producer_thread.join()
production.put(None)
consumer_thread.join()
print("所有线程结束")

生产者生产了数据: 0
消费者消费了数据: 0
生产者生产了数据: 1
消费者消费了数据: 1
生产者生产了数据: 2
消费者消费了数据: 2
生产者生产了数据: 3
消费者消费了数据: 3
生产者生产了数据: 4
消费者消费了数据: 4
所有线程结束


#### threading.local

In [6]:
import threading
import time
import random

# 创建一个线程局部存储
local_data = threading.local()


def thread_task(thread_id):
    # 每个线程有自己的局部数据
    local_data.value = thread_id
    time.sleep(random.randrange(1, 3))
    print(f"线程 {thread_id} 的局部数据: {local_data.value}")


# 创建多个线程并启动
threads = []
for i in range(5):
    t = threading.Thread(target=thread_task, args=(i,))
    threads.append(t)
    t.start()

# 等待所有线程结束
for t in threads:
    t.join()

print("所有线程结束")

线程 1 的局部数据: 1线程 3 的局部数据: 3
线程 4 的局部数据: 4

线程 0 的局部数据: 0线程 2 的局部数据: 2

所有线程结束


#### multiprocessing

---

#### Pipe

#### Manager

#### Queue

In [ ]:
from multiprocessing import Queue

#### Pool

#### Value/Array

#### concurrent.futures

---

#### ThreadPoolExecutor

#### ProcessPoolExecutor

#### async

---

#### async/wait

#### Event loop

#### Queue

In [ ]:
from asyncio import Queue

#### gevent

---

#### Celery

---

#### Ray

---